In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam

# 1. Load the dataset
data_dir = r"C:\Users\mahmo\AppData\Local\Programs\Microsoft VS Code\IMG_CLASSES_GROUPED\Cancer"

# Class label mapping: Melanoma = 0, BCC = 1
classes = {
    "2. Melanoma 15.75k": 0,
    "4. Basal Cell Carcinoma (BCC) 3323": 1
}

images = []
labels = []

# Load and preprocess images
for folder_name, label in classes.items():
    folder_path = os.path.join(data_dir, folder_name)
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        if img_path.lower().endswith((".jpg", ".png", ".jpeg")):
            img = cv2.imread(img_path)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(label)

x = np.array(images)
y = np.array(labels)

# 2. Normalize the images
x = (x - np.mean(x)) / np.std(x)

# 3. Split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# 4. Define the CNN model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(128, 128, 3)))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(LeakyReLU(alpha=0.2))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.2))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

# 5. Compile the model
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 6. Train the model
history = model.fit(
    x_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=64,
    verbose=1
)

# 7. Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test Loss: {test_loss:.4f}")

# 8. Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title("Training and Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

